<a href="https://colab.research.google.com/github/Piece-kun/python-for-DPaV/blob/main/labs/%D0%9B%D0%A01_%D0%9C%D0%B0%D0%BD%D0%B8%D0%BF%D1%83%D0%BB%D1%8F%D1%86%D0%B8%D0%B8_%D1%81_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%87%D0%BD%D1%8B%D0%BC%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Постановка задачи


**Цель работы:** изучение основных способов обработки табличных данных и создания запросов к ним с помощью модуля `pandas`.

**Краткое описание:** в лабораторной работе представлено 8 заданий на сортировку, фильтрацию и группировку данных методами модуля `pandas`. Первые 4 задания являются обязательными. Из оставшихся 4 необходимо выполнить любые два на выбор обучающегося.

# Данные


В папке [Data/lab1](https://drive.google.com/drive/folders/1RxzrWKQC5zqStrKE_3TB9C3Hqzqf-7aJ?usp=share_link) расположено несколько таблиц с обезличенными транзакционными банковскими данными. Все задания лабораторной работы необходимо выполнять по этим данным.

### Таблица ```transactions.csv```
##### **Описание**
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### **Формат данных**

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### **Описание полей**

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком:
  - ```+``` — начисление средств клиенту (приходная транзакция);
  - ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала.


 ### Таблица ```gender_train.csv```

##### **Описание**
Таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### **Формат данных**
```
customer_id,gender
111111,0
111112,1
...
```

##### **Описание полей**
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента.

### Таблица ```tr_mcc_codes.csv```

##### **Описание**
Таблица содержит описание mcc-кодов транзакций.

##### **Формат данных**
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### **Описание полей**
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

### Таблица ```tr_types.csv```

##### **Описание**
Таблица содержит описание типов транзакций.

##### **Формат данных**
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### **Описание полей**
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции.

# Практические задания

In [28]:
import pandas as pd
import numpy as np

## Подготовка

Загрузите и прочитайте данные из таблиц в переменные **tr_mcc_codes, tr_types, transactions** и **gender_train** из одноименных таблиц.

Выведите размеры таблиц и случайные строки из них.


Задания 1–4 нужно выполнить без использования функции `merge` и аналогичных по назначению.


>Допускается для таблицы **transactions** использование не всех записей. При чтении файлов обратите внимание на разделители внутри каждого из файлов – они могут различаться!


## Задание 1

1. Для столбца `tr_type` датафрейма `transactions` выберите произвольные `1000` строк с помощью метода `sample()`.
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец `tr_description` в датафрейме `tr_types`), в которой содержится подстрока `'POS'` или `'ATM'`.


In [29]:
transactions = pd.read_csv(f'/data/lab1/transactions.csv')
tr_types = pd.read_csv(f'/data/lab1/tr_types.csv', sep=';')
tr_mcc_codes = pd.read_csv(f'/data/lab1/tr_mcc_codes.csv', sep=';')
gender_train = pd.read_csv(f'/data/lab1/gender_train.csv')

sample_tr = transactions['tr_type'].sample(n=1000, random_state=42)

tr_type_to_desc = tr_types.set_index('tr_type')['tr_description']
sample_desc = sample_tr.map(tr_type_to_desc)

share_pos_atm = sample_desc.str.contains('POS|ATM', regex=True, na=False).mean()

print('Задание 1')
print(f'Доля POS / ATM транзакций: {share_pos_atm:.4f}')

Задание 1
Доля POS / ATM транзакций: 0.3760


## Задание 2


1. Для столбца `tr_type` датафрейма `transactions` посчитайте частоту встречаемости всех типов транзакций `tr_type` в `transactions`.
2. Выведите датафрейм, содержащий топ-10 транзакций, отсортированных по убыванию частоты встречаемости. Датафрейм должен содержать столбцы `tr_type`, `tr_description` и столбец с частотой встречаемости.

In [30]:
tr_type_freq = transactions['tr_type'].value_counts().reset_index()
tr_type_freq.columns = ['tr_type', 'frequency']

top10_tr_types = tr_type_freq.head(10).copy()
top10_tr_types['tr_description'] = top10_tr_types['tr_type'].map(tr_type_to_desc)


print('Задание 2 — ТОП-10 типов транзакций')
print(top10_tr_types)

Задание 2 — ТОП-10 типов транзакций
   tr_type  frequency                                     tr_description
0     1010    1589734                              Покупка. POS ТУ СБ РФ
1     2010    1040713              Выдача наличных в АТМ Сбербанк России
2     1030     970994                     Оплата услуги. Банкоматы СБ РФ
3     1110     928154                             Покупка. POS ТУ Россия
4     7070     691896  Перевод на карту (с карты) через Мобильный бан...
5     2370     376521  Списание с карты на карту по операции <перевод...
6     7010     258088       Взнос наличных через АТМ (в своем тер.банке)
7     7030     155200  Перевод на карту (с карты) через АТМ (в предел...
8     1100     117870                                Покупка. ТУ  Россия
9     7071      96146  Перевод на карту (с карты) через Мобильный бан...


## Задание 3
1. В датафрейме `transactions` найдите клиента с максимальной суммой приходов на карту.
2. В датафрейме `transactions` найдите клиента с максимальной суммой расходов по карте.
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов.

In [31]:
income = transactions[transactions['amount'] > 0] \
    .groupby('customer_id')['amount'].sum()

expense = transactions[transactions['amount'] < 0] \
    .groupby('customer_id')['amount'].sum()

max_income_customer = income.idxmax()
max_expense_customer = expense.idxmin()

income_sum = income[max_income_customer]
expense_sum = expense[max_expense_customer]

diff_module = abs(income_sum + expense_sum)

print('Задание 3')
print(f'Клиент с макс. доходами: {max_income_customer}')
print(f'Клиент с макс. расходами: {max_expense_customer}')
print(f'Модуль разницы: {diff_module:.2f}')

Задание 3
Клиент с макс. доходами: 48281795
Клиент с макс. расходами: 48281795
Модуль разницы: 365148.44


## Задание 4
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций из топ-10 в Задании 2.
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций, совершенных клиентами из Задания 3.

In [32]:
print('Задание 4.1 — Среднее и медиана (ТОП-10 типов)')

top10_ids = top10_tr_types['tr_type']

stats_top10 = (
    transactions[transactions['tr_type'].isin(top10_ids)]
    .groupby('tr_type')['amount']
    .agg(['mean', 'median'])
)

print(stats_top10)
print('-' * 60)

print('Задание 4.2 — Клиенты из Задания 3')

stats_clients = (
    transactions[transactions['customer_id'].isin([max_income_customer, max_expense_customer])]
    .groupby('tr_type')['amount']
    .agg(['mean', 'median'])
)

print(stats_clients)

Задание 4.1 — Среднее и медиана (ТОП-10 типов)
                  mean    median
tr_type                         
1010     -17056.263411  -6733.26
1030      -4700.005100  -2245.92
1100     -39390.923622  -8983.66
1110     -26676.998508  -9365.47
2010    -135584.063351 -44918.32
2370    -197634.813726 -43795.36
7010     259527.366684  93205.50
7030     135919.508463  22459.16
7070     100736.485505  14284.02
7071     137771.552874  32341.19
------------------------------------------------------------
Задание 4.2 — Клиенты из Задания 3
                 mean        median
tr_type                            
1030    -6.737745e+03 -6.737745e+03
2011    -4.239080e+06 -4.491832e+06
2330    -6.737747e+06 -6.737747e+06
2331    -4.786795e+06 -4.734390e+06
2370    -1.470826e+07 -1.122958e+07
4031    -1.684000e+01 -1.684000e+01
4051    -2.245916e+04 -2.245916e+04
7011     1.684998e+06  1.684437e+06
7030     4.321784e+06  4.491832e+06
7070     1.475677e+07  1.122958e+07
7074     9.810191e+06  1.1229

## Подготовка для заданий 5–8

>**Из заданий 5-8 нужно выполнить два любых на выбор.**

Соедините датафрейм `transactions` со всеми остальными таблицами (`tr_mcc_codes`, `tr_types`, `gender_train`). Причем объединение с таблицей `gender_train` необходимо выполнить с помощью `left join`, а с оставшимися датафреймами – через `inner`.
После получения общей таблицы `gender_train`, `tr_types` и `tr_mcc_codes` можно удалить. В результате соединения датафреймов должно получиться `999584` строки.

In [33]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(6846580, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата – отрицательное значение в столбце `amount`).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин.

>Обратите внимание, что для вычисления модуля разности не требуется точных знаний о том,
каким значением в таблице обозначены мужчины, а каким – женщины.

In [34]:
expenses = transactions[transactions['amount'] < 0]
income = transactions[transactions['amount'] > 0]

mean_expense_by_gender = expenses.groupby('gender')['amount'].mean()
mean_income_by_gender = income.groupby('gender')['amount'].mean()

expense_diff = abs(mean_expense_by_gender.iloc[0] - mean_expense_by_gender.iloc[1])
income_diff = abs(mean_income_by_gender.iloc[0] - mean_income_by_gender.iloc[1])

print('Задание 5')
print(f'Разница средних трат: {expense_diff:.2f}')
print(f'Разница средних поступлений: {income_diff:.2f}')

Задание 5
Разница средних трат: 24748.04
Разница средних поступлений: 65084.17


## Задание 6

1. Для каждого типа транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу `amount`) отдельно для мужчин и женщин (назовите ее `max_income`). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям `max_income`.
2. Среди типов транзакций, найденных в пункте 1, выделите те, которые встречаются одновременно и у мужчин, и у женщин.

In [35]:
positive_income = transactions[transactions['amount'] > 0]

max_income = (
    positive_income
    .groupby(['gender', 'tr_type'])['amount']
    .max()
    .reset_index(name='max_income')
)

men_top10 = (
    max_income[max_income['gender'] == max_income['gender'].unique()[0]]
    .nsmallest(10, 'max_income')
)

women_top10 = (
    max_income[max_income['gender'] == max_income['gender'].unique()[1]]
    .nsmallest(10, 'max_income')
)

common_tr_types = set(men_top10['tr_type']) & set(women_top10['tr_type'])

print('Задание 6')
print('Общие типы транзакций для мужчин и женщин:')
print(common_tr_types)

Задание 6
Общие типы транзакций для мужчин и женщин:
{4100, 4071, 4110, 4210, 4051, 8146}


## Задание 7

1. Найдите суммы затрат по каждой категории (`mcc`) для мужчин и для женщин.
2. Найдите топ-10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc-кодов).

In [36]:
# PUT YOUR CODE HERE

## Задание 8

1. Из поля `tr_datetime` выделите час `tr_hour`, в который произошла транзакция, как первые 2 цифры до `":"`.
2. Посчитайте количество транзакций со значением в столбце `amount` строго меньше `0` в ночное время для мужчин и женщин. Ночное время – это промежуток с 00:00 по 06:00 часов.

In [37]:
# PUT YOUR CODE HERE

# Порядок защиты работы

**Для защиты работы необходимо:**
1. Предоставить результаты выполнения 6 заданий в виде ссылки на Google Colab.
2. Выполнить дополнительное задание в процессе сдачи работы. Дополнительное задание представляет собой запрос к данным лабораторной работы, по сложности аналогичный тем, что требовалось выполнить в рамках работы.

**Примеры дополнительных заданий:**
1. Найдите категорию транзакций, в которой совершается больше всего трат в ночное время (промежуток с 00:00 по 06:00). Выведите название категории и среднюю сумму трат в ней.
2. Найдите 3 категории транзакций, в которых средние траты мужчин и женщин различаются меньше всего.
3. Найдите терминал, через который проходит наибольшее число транзакций. Выведите идентификаторы 5 пользователей с наибольшей суммой отрицательных транзакций в этом терминале.

# Примечание

Рекомендуется выполнение заданий двумя способами – с помощью `pandas` и с помощью `SQL` с использованием модуля `sqlite3`.

# Дополнительные материалы

1. Pandas. User Guide [Электронный ресурс]. URL: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html (дата обращения: 14.03.2024).
2. МакКинни, У. Python и анализ данных [Текст] / У. МакКинни ; пер. с англ. Слинкиной А. А. — Москва : ДМК-Пресс, 2023. — 536 с.
3. Хейдт, М. Изучаем pandas [Текст] / М. Хейдт. — Москва : ДМК Пресс, 2019. — 700 с.